In [10]:
import sys
import os
import numpy as np
import grand.dataio.root_files as gdr
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

################################################################################
##Functions

################################################################################
##Main
rootsim_path = '/home/grand/sim2root/Common/sim_nosite_19000101__0001/trunefieldsim_0-0_L0_0000.root'


event_files = gdr.get_file_event(rootsim_path) #open the root file with all events
nevent = event_files.get_nb_events() #get the total number of events


#for i in range(nevent-1):#loop over all events
for i in range(nevent):#loop over all events
    print(f"->{(i+1)/nevent*100:.1f}%")

    event_files.load_event_idx(i)
    zenith = event_files.tt_shower.zenith
    azimuth = event_files.tt_shower.azimuth
    energy_primary = event_files.tt_shower.energy_primary
    xmax = event_files.tt_shower.xmax_pos_shc #Xmax location in shower core frame (because doesn't work in detector frame!)
    xmax_dist = np.linalg.norm(xmax) #Xmax distance to shower core
    shower_core = event_files.tt_shower.shower_core_pos #in detector frame
    coreAlt = event_files.tt_shower.core_alt #??
    MagneticField = event_files.tt_shower.magnetic_field
    primary = event_files.tt_shower.primary_type
    xmax_gram = event_files.tt_shower.xmax_grams
    energy_em = event_files.tt_shower.energy_em
    energy_unit = 'GeV'# could not find corresping field in root files

    print(f"Zenith = {zenith:.1f}°, Azimuth = {azimuth:.1f}°, Primary energy = {energy_primary:.3e} GeV, EM energy = {energy_em:.3e} GeV")
    print(f"Xmax location ({xmax[0]:.2f}, {xmax[1]:.2f}, {xmax[2]:.2f}), Xmax grammage = {xmax_gram:.3f} g/cm^2, Primary type {primary}")
    print(f"Shower core location ({shower_core[0]:.2f}, {shower_core[1]:.2f}, {shower_core[2]:.2f}), core altitude = {coreAlt:.2f} m")


    ef3d = event_files.get_obj_handling3dtraces()
    peakT, peakA = ef3d.get_tmax_vmax(hilbert=True)
    #peakT, peakA = ef3d.get_tmax_vmax()
    xant, yant, zant = ef3d.network.du_pos.T
    eventID = int(ef3d.name.split("EVT_NB=")[1].split(",")[0]) 
    #eventID = int(ef3d.name.split("EVT_NB=")[0].split(",")[0]) 
    runID = int(ef3d.name.split("RUN_NB=")[1])

    coreAlt = 1264. #m -> not filled in root files...
    zant +=coreAlt
    shower_core[2] += coreAlt

    xmax_grand = xmax + shower_core #detector frame
    xmax_grand[2] -= coreAlt #because zxmax is laready from sea level...
    k_shower = np.array([np.cos(azimuth*np.pi/180.)*np.sin(zenith*np.pi/180),np.sin(azimuth*np.pi/180.)*np.sin(zenith*np.pi/180), np.cos(zenith*np.pi/180)]).T

    print(event_files.tt_run.du_geoid)
    print(event_files.tt_run.origin_geoid)
    print(event_files.tt_run.du_xyz[2])

    ############
    ##Check plots
    fa, ax = plt.subplots(subplot_kw={'projection': '3d'})

    fa.suptitle(f"Run:{runID:d}, event:{eventID:d}\n"+f"Zenith = {zenith:.1f}°, Azimuth = {azimuth:.1f}°, Primary energy = {energy_primary:.3e} GeV\n"+f"Xmax location ({xmax[0]:.2f}, {xmax[1]:.2f}, {xmax[2]:.2f})")
    ax.set_xlabel(r"$\rm x\ (km)$")
    ax.set_ylabel(r"$\rm y\ (km)$")
    ax.set_zlabel(r"$\rm z\ (m)$")

    a = ax.scatter(xant/1.e3, yant/1.e3, zant, c=peakT)
    ax.scatter(shower_core[0]/1.e3, shower_core[1]/1.e3, shower_core[2], color='red')
    cbara = fa.colorbar(a, ax=ax)
    cbara.set_label(r"$\rm Peak\ Time\ (ns)$")


    fb, (b1x, b2x) = plt.subplots(1,2)

    fb.suptitle(f"Zenith = {zenith:.1f}°, Azimuth = {azimuth:.1f}°, Primary energy = {energy_primary:.3e} GeV\n"+f"Xmax location ({xmax[0]:.2f}, {xmax[1]:.2f}, {xmax[2]:.2f})")
    b1x.set_xlabel(r"$\rm x\ (km)$")
    b2x.set_xlabel(r"$\rm y\ (km)$")
    b1x.set_ylabel(r"$\rm z\ (m)$")
    #b2x.set_ylabel(r"$\rm z\ (m)$")

    b1x.scatter(xant/1.e3, zant, c=peakT)
    b2x.scatter(yant/1.e3, zant, c=peakT)
    b1x.scatter(shower_core[0]/1.e3, shower_core[2], color='red', label='core')
    b2x.scatter(shower_core[1]/1.e3, shower_core[2], color='red')
    b1x.scatter(xmax_grand[0]/1.e3, xmax_grand[2], color='red', label='xmax')
    b2x.scatter(xmax_grand[1]/1.e3, xmax_grand[2], color='red')
    b1x.plot([xmax_grand[0]/1.e3, xmax_grand[0]/1.e3-k_shower[0]*xmax_dist/1.e3], [xmax_grand[2], xmax_grand[2]-k_shower[2]*xmax_dist])
    b2x.plot([xmax_grand[1]/1.e3, xmax_grand[1]/1.e3-k_shower[1]*xmax_dist/1.e3], [xmax_grand[2], xmax_grand[2]-k_shower[2]*xmax_dist], label='(theta, phi) inputs')
    b1x.legend()
    b2x.legend()
      
    plt.show()
    ############

print("-> Done")

File /home/grand/sim2root/Common/sim_nosite_19000101__0001/trunefieldsim_0-0_L0_0000.root doesn't content TTree teventefield or teventvoltage. It contains ['trunefieldsim'].


AssertionError: 

In [10]:
range(nevent)

range(0, 1)

In [2]:
import sys
import os
import numpy as np
import grand.dataio.root_files as gdr
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
gdr.get_ttree_in_file('/home/grand/sim2root/CoREASRawRoot/gr_Coreas_100141.root')

['trun', 'trunshowersim', 'trunefieldsim', 'tshower', 'tshowersim', 'tefield']

In [3]:
gdr.get_file_event('/home/grand/sim2root/CoREASRawRoot/gr_Coreas_100141.root')